# Baixa e limpa os dados

In [ ]:
# Geospatial data processing
import geopandas as gpd

# The star of the show: city2graph for transportation network analysis
import city2graph as c2g

# Configure matplotlib for publication-quality visualizations
plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 10
plt.style.use('ggplot')
#plt.style.use('default')  # Clean default style instead of ggplot

print("All dependencies loaded successfully!")
print(f"city2graph version: {c2g.__version__ if hasattr(c2g, '__version__') else 'development'}")

All dependencies loaded successfully!
city2graph version: 0.2.3



## 1 Download de dados (segments, connector, building) do OvertureMaps

In [ ]:
# Caixa delimitadora (Bounding Box, BBOX) do municípios
# [min_lon, min_lat, max_lon, max_lat]
# bbox3km_zonaUrbana = [-42.863200,-5.259269, -42.626205, -4.929381] # BBOX do buffer de 3km da zona urbana
bbox = [-42.98,-5.59,-42.60,-4.79] # BBOX de Teresina

path_bruto = "../dados/brutos/overturemaps/"

In [ ]:
overture_data = c2g.load_overture_data(
    area=bbox,
    types=["segment", "connector"],
    output_dir=path_bruto,
    prefix="teresina_bbox_",
    save_to_file=True,
    return_data=False,
)

print(f"O download dos dados ({types}) foi \nconcluído e salvo no computador com sucesso.")

O download dos dados ['segment', 'connector'] foi concluído.


In [ ]:
segment_gdf = overture_data['segment']
connector_gdf = overture_data['connector']

In [ ]:
print(f'\nSegmentos: {segment_gdf.shape[0]} linhas e {segment_gdf.shape[1]} colunas')


Segmentos: 66486 linhas e 20 colunas


In [ ]:
print(f'\nConectores {connector_gdf.shape[0]} linhas e {connector_gdf.shape[1]} colunas')


Conectores 49664 linhas e 4 colunas


In [ ]:
segment_gdf.head(2)

,id,version,sources,subtype,class,names,connectors,routes,subclass_rules,access_restrictions,level_rules,destinations,prohibited_transitions,rail_flags,road_surface,road_flags,speed_limits,width_rules,subclass,geometry
0,6d775c41-5cf2-493e-b402-cfcbd8972262,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,unclassified,NaN,[{'connector_id': 'eaaeee83-1ec8-4c2f-a46e-dde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'unpaved', 'between': None}]",NaN,NaN,NaN,NaN,"LINESTRING (-42.98426 -5.34934, -42.98401 -5.3..."
1,aec4047f-e9b3-4c2b-ba48-c1c065ffc97e,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,unclassified,NaN,[{'connector_id': '89a3c9a6-e65e-4102-888f-499...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'unpaved', 'between': None}]",NaN,NaN,NaN,NaN,"LINESTRING (-42.97465 -5.28342, -42.97852 -5.2..."


In [ ]:
connector_gdf.head(3)

,id,version,sources,geometry
0,1a9f6ba4-85ea-4623-b796-403dac35c312,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.97852 -5.34666)
1,c32213b5-fe58-4ca9-a910-32aec6ff0181,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.97931 -5.29226)
2,3d80bce1-d842-459a-9195-f849da94e6fe,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.95956 -5.30976)


In [ ]:
# -------------------------------------- #
# TIPOS DE VIA
# -------------------------------------- #
print(f'\n foram encontrados {len(segment_gdf['subtype'].unique().tolist())} tipos de segmentos:\n{segment_gdf['subtype'].unique().tolist()}')


 foram encontrados 2 tipos de segmentos:
['road', 'rail']


In [ ]:
# --------------------------------------
# CLASSE DAS VIA
# --------------------------------------
print(f'\n foram encontrados {len(segment_gdf['class'].unique().tolist())} classe de segmentos:\n{segment_gdf['class'].unique().tolist()}')


 foram encontrados 17 classe de segmentos:
['path', 'unclassified', 'tertiary', 'track', 'primary', 'residential', 'trunk', 'service', 'footway', 'unknown', 'living_street', 'secondary', 'pedestrian', 'cycleway', 'steps', 'standard_gauge', 'light_rail']


In [ ]:
# --------------------------------------
# SUBCLASSE DAS VIA
# --------------------------------------
print(f'\n foram encontrados {len(segment_gdf['subclass'].unique().tolist())} classe de segmentos:\n{segment_gdf['subclass'].unique().tolist()}')


 foram encontrados 8 classe de segmentos:
[nan, 'link', 'crosswalk', 'driveway', 'alley', 'sidewalk', 'parking_aisle', 'cycle_crossing']


In [ ]:
segment_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# Garantir projeção métrica (fundamental para Alpha Shapes funcionarem bem)
if segment_gdf.crs.to_string() != "EPSG:31983":
    print(f"Segmentos -> CRS Fonte = {segment_gdf.crs.to_string()}. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...")
    gdf_segments_utm = segment_gdf.to_crs(epsg=31983)

if connector_gdf.crs.to_string() != "EPSG:31983":
    print(f"Conectores -> CRS Fonte = {connector_gdf.crs.to_string()}. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...")
    gdf_conectores_utm = connector_gdf.to_crs(epsg=31983)

Segmentos -> CRS Fonte = EPSG:4326. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...
Conectores -> CRS Fonte = EPSG:4326. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...
